In [ ]:
import requests
from bs4 import BeautifulSoup
import os

In [ ]:
url = 'https://www.inskysolutions.com/'
response = requests.get(url)

In [ ]:
# if response.status_code == 200:
if response.ok:
    html = response.content
    print(html[:100])
    soup = BeautifulSoup(html, 'html.parser')


### CILJ
- dobiti što više tekstova 
    - zanimaju nas svi opisni tekstovi (o pojedinim produktima, o tvrtki, o integracijama, rješenjima itd...)

### KAKO 
- S dev toolsima inspectamo glavnu stranicu i tražimo najbolji način za doći do tekstova
    - tražimo linkove za druge pageve gdje ima više teksta
    - pogledajte da već nakon prvih par divova unutar <form name> možemo doći do nav bara koji ima id='navigation'!!

In [ ]:
navigation = soup.find_all('nav', id='navigation')
print(type(navigation))
print(len(navigation))

In [ ]:
# ono što sada možemo je iterirati kroz bs4.element.ResultSet kao što iteriramo kroz listu 
# https://stackoverflow.com/questions/36076052/beautifulsoup-find-all-on-bs4-element-resultset-object-or-list

buttons = []
for element in navigation:
    buttons.append(element.find_all('a')) # <class 'bs4.element.Tag'>
    # print(nav_button.text)
print(buttons)
print(len(buttons))
print(len(buttons[0]))


In [ ]:
### sad vidimo da možemo izvući linkove iz gornje liste
### ali nam ne trebaju svi nego samo oni koji imaju neki "tekst" (ima puno praznih 'a' elemenata) po hrefom
### to nije isto kao i .text jer bismo s tekstom dobili ono što je pod TITLE

# https://stackoverflow.com/questions/57395509/get-item-from-bs4-element-tag
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag

linkovi = []
kategorije = []
for element in navigation:
    tags = element.find_all('a') 
    for tag in tags:
        link = tag.get('href')
        ### sad imamo stringove koji predstavljaju linkove na druge 'pages' na ovoj web stranici
        ### s obzirom da su neki 'anchori' na main pageu, njih zaobilazimo
        # print(tag.text)
        if link != '#':
            linkovi.append(link)
            kategorije.append(tag.text)
        
print(kategorije)
print(linkovi)
print(len(linkovi))

assert len(linkovi) == len(kategorije)

### INSPECTAMO JOŠ JEDAN PAGE OD OVIH ČIJE SMO LINKOVE SADA IZGENERIRALI

In [ ]:
### svi tekstovi se nalaze u divovima unutar <main id='pageContainer'> elementa
### ponavljamo sve ispočetka 
    ### primijetite kako bi se ovaj proces sada trebao pretvoriti u funkcije jer se ponavljamo :) 
page_url = 'https://www.inskysolutions.com/tourisminsky.aspx'
response = requests.get(page_url)
if response.ok:
    html = response.content
    print(html[:100])
    soup = BeautifulSoup(html, 'html.parser')

divs = soup.body.form.main.find_all('div')
print(len(divs))

articles = [div.find('article') for div in divs]
print(len(articles))

paragraphs = []
for element in articles:
    if element is not None:
        # print(type(element)) 
        # ako je tag, možemo pozivati sve atribute koje vidimo na https://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag

        ### ako želimo, možemo spremiti ove tekstove ili možemo raditi dodatne filtere s obzirom na elemente, id-eve ili klase
        # print(element.text)
        paragraphs.append(element.text.strip())
print(len(paragraphs))
print(paragraphs)

### PRETPOSTAVKA
- trebalo bi biti dovoljno 'inspectati' jedan page, a drugi bi trebali imati istu strukturu
- ako složimo url-ove pomoću glavnog i svih linkova iz napravljene liste, možemo kroz petlju iterirati i izbaciti van željene tekstove/podatke

In [ ]:
### koji modul možemo koristiti za spajanje putanja? :) 
### odmah možete vidjeti i zašto to nije najbolja opcija - https://stackoverflow.com/questions/1793261/how-to-join-components-of-a-path-when-you-are-constructing-a-url-in-python
base_url = 'https://www.inskysolutions.com/'
for l in linkovi:
    link = os.path.join(base_url,l[1:])
    print(link)

In [ ]:
page_texts = []
urls = []

base_url = 'https://www.inskysolutions.com/'
for l in linkovi:
    link = os.path.join(base_url,l[1:])
    urls.append(link)

    response = requests.get(link)
    if response.ok:
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')

        ### ako dobijem error, želim da mi kod samo preskoči taj cijeli soup object/trenutnu stranicu
        ### https://stackoverflow.com/questions/19522990/python-catch-exception-and-continue-try-block
        ### ovo je loša praksa, inače, jer ako već hvatate neke errore/exceptione, onda bi bilo oke znati što je to :)
        try:
            divs = soup.body.form.main.find_all('div')

            articles = [div.find('article') for div in divs]
            # print(len(articles))

            paragraphs = []
            for element in articles:
                if element is not None:
                    # print(type(element)) 
                    # ako je tag, možemo pozivati sve atribute koje vidimo na https://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag

                    ### ako želimo, možemo spremiti ove tekstove ili možemo raditi dodatne filtere s obzirom na elemente, id-eve ili klase
                    # print(element.text)
                    paragraphs.append(element.text.strip())
            # print(len(paragraphs))
            # print(paragraphs)
        except Exception:
            continue

    page_texts.append(paragraphs)
print(len(page_texts))
# print(page_texts)

In [ ]:
assert len(page_texts) == len(kategorije)

### Dovoljno dobro? 
...


------------------------------------

In [ ]:
### saveamo naše liste u csv file koji onda dalje možemo "exploreati"

import csv
csv_filename = 'data_v1.csv'

rows = zip(kategorije, urls, page_texts)
if not os.path.isfile(os.path.join(os.getcwd(), csv_filename)):
    with open(os.path.join(os.getcwd(), csv_filename), "w") as f:
        writer = csv.writer(f)
        for row in rows:
            writer.writerow(row)
else:
    print("the file already exists!!!")